# Лекция: Обучение с подкреплением

Будем говорить про Reinforcement Learning (RL) больше в контексте LLM, поскольку для них есть целый этап обучения связанный с этим подходом, называемый Alignment.

Начнем с проблематики, а именно с задачи суммаризации текста. Если для базовых задач мы легко вводили метрики качество, то тут ситуация иная и как оценить качество суммаризации не понятно. Конечно, есть метрика ROUGE (Recall-Oriented Understudy for Gisting Evaluation) - она считается как число слов из целевого ответа входящих в ответ модели (ROUGE-1 для униграм ROUGE-2 для биграм). Однако, как было выявлено экспериментально, эта метрика не выражает действительного качеств суммаризации и имеет асимптоту даже для самых совершенных алгоритмов.

Поговорим о классической дилемме из ML - мы обучаем модель (то есть используем в качестве функции потерь) не тоже самое, на чем далее проверяем качество. Победить эту дилемму и оптимизировать именно то, что нужно как раз позволяют методы RL

Для моделей в парадигме RL вводят награду, будем использовать оригинальный термин - Reward. OpenAI учили модель играть в Dota 2 записывали различные таблицы - но такой метод бы плохо масштабировался. Важно Reward функции проектировать аккуратно, иначе после оптимизации модель будет делать не то, что нужно, а "взломает" наш reward

Сделать действительно хороший reward сложно, поэтому можно использовать человека для этого и, чтобы упростить ему жизнь, можно давать два варианта и выбора меньше ними.

- Pretrain - просто на больших объемах данных
- Finetune (SFT) - используются датасеты инструкции (демонстрации решения задач)
- RLHF - обучаем Reward модель давать скалярную оценку на данных от разметки пользователей, затем оптимизируем генерацию чтобы максимизировать оценку генеративной модели

Схема и описание этапов обучения модели [ChatGPT](https://openai.com/index/chatgpt/).

Поговорим именно про последний этап и для этого введем основные термины.

Policy Model - большая генеративная модель (например, после SFT), которую мы в итоге хотим обучить.

Reward Model - относительно небольшая модель, которую мы обучаем на парах сравнения, чтобы она предсказывала скалярное reward.

Reward модель - должна научиться понимать предпочтения людей в виде скаляра reward (или score), что само по себе очень интересная задача. Прочисть что было придумано в этой области можно в обзорной [статье](https://arxiv.org/abs/2504.12328)

Пусть у нас есть два ответа на вопросы - победивший и проигравший. Подадим на вход некоторой модели (можно и породившей эти ответы, но чаще просто трансформер меньшего размера, близкого по архитектуре к policy модели), каждый из них вместе с первоначальным вопросом. Для победившего $ s_win = r_\theta(x, y_win) $ и для проигравшего 
$ s_lose = r_\theta(x, y_lose) $. Модель как обычно предскажет вектор, но на этот раз найдем его проекцию не на словарь, а на скаляр - так получаем reward. Затем применим pairwise ranking loss который использует не абсолютные величины, а их относительные значения (у победившего reward больше, чем у проигравшего) $ -log(\sigma(s_win - s_lose)) $

Поскольку награда в RL выдается за действие, а действие переводит в новое состояние, то это хорошо соотносится с авторегрессионной природой LLM

Введем такое понятие как value функция (модель критик) - она указывает сколько reward'а мы можем заработать

Считаем advantage это разница в value предсказанных value-функцией и фактически полученным reward между двумя соседними состояниями. Это мера того, насколько конкретное действие (токен) оказалось лучше или хуже, чем "среднее" по политике в данном состоянии.

Пусть есть 10к-100к промптов, для каждого из них:
- Генерируем ответ текущей моделью
- Оценивать его Reward моделью
- Оценивать его моделью критиком, то есть предсказывать value значения
- Дообучаться для улучшения общего reward и отдельных advantage

Промптов модно взять и меньше, можно и сотни тысячи, они могут быть сгенерированы другой моделью и т.д.

Reward модель при этом дистиллирует понимание прекрасного людей в отношении ответов именно на эти промпты.

Reward модель однако все еще может закреплять не очень хорошие практики ответов модели.

Есть набор промптов, подаем их в исходную модель (например после SFT) и в нашу текущую модель, которую мы обучаем.

Обе модели делаю генерации и для каждого токена замеряем распределение вероятностей и затем считаем дивергенцию Кульбака-Лейблера $ L_{KL} $ между двумя распределениями вероятностей - добавляем это как некоторый регуляризационный член в оптимизируемую функцию. Это не даст очень сильно угодить от генерации правдоподобного текста, который хорошо умеет делать SFT модель.

$ L_{PPO} = L_{RL} - \beta \cdot L_{KL} + \gamma \cdot L_{Value} $

$ L_{RL} $ - ожидаемый reward от новой политики, считается через advantage, который показывает, насколько действие (токен) в данном контексте было лучше среднего.

$ L_{RL} = \hat{\mathbb{E}}t [ \frac{\pi\theta(y_t | x, y_{<t})}{\pi_{\text{old}}(y_t | x, y_{<t})} \cdot A_t ] $

$ L_{Value} $ - ошибка предсказания value-функции.

Весь RLHF этап по сути не меняет знаний модель, не добавляет новых, а лишь позволяет разблокировать возможность пользоваться уже имеющимися знаниями для решения конкретных задач - то есть происходит выравнивание с ожиданиями человека. Проведение этого [этапа](https://arxiv.org/abs/2203.02155) на промптах исключительно на английском языке учит модель не английскому, а следованию инструкциям, причем и на других языках.

Это был метод [Proximal Policy Optimization](https://arxiv.org/abs/1707.06347) (PPO) - это on-policy метод, он должен стыкаваться с текущим графом знаний модели - поэтому опастно делать RLHF открытой модели использую датасет сгенерированный какой-либо проприетарной (разные графы знаний - модель ухудшится)

Есть библиотека [TLR](https://huggingface.co/docs/trl/main/en/index) - Transformer Reinforcement Learning - в ней предоставляются методы SFT, GRPO, DPO, построения Reward Model и не только. Конкретный пример использования [GRPO](https://huggingface.co/learn/cookbook/en/fine_tuning_llm_grpo_trl)

Существует также [Direct Preference Optimization](https://arxiv.org/abs/2305.18290) или DPO, который позволяет напрямую оптимизировать модель под предпочтения пользователей, без обучения отдельной reward-модели.

Функция потерь DPO выглядит следующим образом:

$L_{DPO} = -\hat{\mathbb{E}} [ \log \sigma ( \beta \log \frac{\pi_\theta(y_{win} | x)}{\pi_{\text{ref}}(y_{win} | x)} - \beta \log \frac{\pi_\theta(y_{lose} | x)}{\pi_{\text{ref}}(y_{lose} | x)} ) ]$

в этой формуле:

$\pi_\theta(y | x)$ - вероятность генерации ответа $y$ на промпт $x$ текущей обучаемой моделью.

$\pi_{\text{ref}}(y | x)$ - вероятность генерации ответа $y$ на промпт $x$ исходной эталонной моделью (например, после SFT) и она заморожена.

$\log \frac{\pi_\theta(y | x)}{\pi_{\text{ref}}(y | x)}$ - это и есть неявная reward, она показывает, насколько текущая модель "предпочитает" этот ответ по сравнению с эталонной.

$\beta$ — параметр, регулирующий силу отклонения от эталонной модели (аналогичен параметру $\beta$ для KL-штрафа в PPO).

Вся конструкция внутри $ \log \sigma $ - это разница в reward между выигравшим и проигравшим ответами. Мы просто максимизируем вероятность того, что выигравший ответ имеет большую неявную reward, чем проигравший.

Пример в библиотеке [TLR](https://huggingface.co/docs/trl/main/en/dpo_trainer)

GRPO - это еще один вариант on-policy оптимизации для LLM. Главное отличие от классического PPO тут в том, что нет отдельной value-функции, вместо нее в GRPO генерируется для каждого промпта группа $K$ кандидатов (то есть не только победитель и проигравший, а целый конкурс ответов), оценивается их качество (обычно reward-моделью) и строится advantage внутри этой группы (т.е. сравнивает ответы друг с другом).

Формально, GRPO записывается примерно следующим образом:

Пусть для промпта $x_i$ сгенерировали группу $j=1..K$ кандидатов ответов $y_{ij}$. Каждому сопоставлен reward $s_{ij}=r(x_i,y_{ij})$ (reward-моделью).
Внутри группы выбираем baseline, например, ответ со средней оценкой или через leave-one-out и считаем advantage $ A_{ij} = s_{ij} - b_i $. ДалееНаходим отношение вероятностей (importance ratio):

$$
r_{ij}(\theta) = \frac{\pi_\theta(y_{ij}\mid x_i)}{\pi_{\text{ref}}(y_{ij}\mid x_i)}.
$$

4. **Surrogate (PPO-style) objective** — GRPO использует PPO-подобную суррогатную функцию:

$$
L_{\text{GRPO}}(\theta) = -\mathbb{E}_{i,j}\Big[ \min\big( r_{ij}(\theta) A_{ij},\ \operatorname{clip}(r_{ij}(\theta),1-\epsilon,1+\epsilon) A_{ij}\big)\Big] + \lambda\,\mathbb{E}_i\big[\mathrm{KL}(\pi_{\text{ref}}(\cdot|x_i)\Vert\pi_\theta(\cdot|x_i))\big].
$$

Градиент вычисляется как обычно через $\nabla_\theta \log\pi_\theta$ и взвешивается преимуществом $A_{ij}$ (с учётом клиппинга). Если $A_{ij}>0$ (ответ лучше baseline), градиент направлен повысить $\pi_\theta(y_{ij}|x_i)$ — через $\nabla_\theta\log\pi_\theta$. Если $A_{ij}<0$ — наоборот. Клиппинг гарантирует, что обновления не будут слишком большими (по аналогии с PPO). Поскольку baseline берётся по группе, мы избегаем обучения отдельной value-функции, но дисперсия оценки преимущества может быть выше/ниже в зависимости от $K$ и качества reward-модели.

Пример использования GRPO в библиотеке [TLR](https://huggingface.co/docs/trl/main/en/grpo_trainer).

В качестве метода RLHF в DeepSeek-R1 используется как раз GRPO, представленный в статье [DeepSeekMath](https://arxiv.org/abs/2402.03300). Подробнее про обучение модели рассуждениям можно узнать в статье про [DeepSeek-R1](https://arxiv.org/abs/2501.12948).

Можно мыслить в контексте того, что модель - это просто некая форма хранения данных. Отельным вопросом является то, происходит ли при этом сжатие

Арифметическое кодирование

Колмогоровская сложность

Принцип минимальной длины описания

За счет наблюдения большого числа паттернов информации, трансформер обладает лучшими свойствами сжатия чем специализированные алгоритмы

Есть даже метрика, связанная с этим - Bits-per-byte, а также [статьи](https://arxiv.org/abs/2309.10668)


Dense Attention

Sparse Attention (формулы)
От n*n до n sqrt(n). Но его в итоге редко применяют


Flash Attention - вариант аппаратного ускорения. В стандартной реализация self-attention использует матрицу $ L \times L $ (где $L$ — длина последовательности) - ее тяжело держать в памяти, но операции проводить с ней можно. Идея в том чтобы делать даже больше вычислений, но меньше перекладывания из памяти (в GPU есть HBM (High-Bandwidth Memory) - большая и долгая, SRAM - маленькая и быстрая). Для этого матрицы Q, K, V разбиваются на T блоков по разным измерениям, загружаем такие блоки в SRAM, делаем все операции ($ \text{softmax} (Q_i @ K_j^T) V_j $) и получаем кусочек выходной матрицы. Однако, считаем статистики softmax между такими шагами и корректируем значения итоговой матрицы при необходимости. Вторая версия этого алгоритма лучше распределяет работу между несколькими ядрами GPU, добавляет параллельность по последовательности, разбивает на блоки матрицы K и V, а не Q, как в первой версии.

[MoE](https://huggingface.co/blog/moe) - это паттерн сокращения "активных" весов модели. Как было ранее упомянуто, большая часть весов модели это полносвязные слои (feed-forward network слои, FFN), идущие после self-attention, поэтому работать будем именно с ними. В модели каждый стандартный FFN-блок заменяют на набор 
$ E $ (экспертов) (малых FFN) и на каждый токен теперь активируется только небольшой набор экспертов (top-k). Перед применением весов экспертов ставится gate network или router, который определяет какие токены будут поданы в какого эксперта - это простой линейный слой с логитами по экспертам, к его выходу добавляют softmax и получают веса экспертов для токена $ w_{t,e} $, они используются при агрегации ответов экспертов. Обучение router'а происходит просто за счет backprop модели, лишь с [оговоркой](https://arxiv.org/abs/1701.06538) на добавление шума при операции top-k или других доработках, распределяющих нагрузку на экспертов более равномерно.

[Обзор](https://arxiv.org/pdf/2402.19473) методов видов RAG





AlphaEvolve — это агентский эволюционный фреймворк, который сочетает в себе большие языковые модели (LLM) и эволюционные алгоритмы для автоматического открытия и оптимизации решений сложных задач. Его ключевая идея — не просто сгенерировать код один раз, а итеративно улучшать его, создавая "популяцию" решений, скрещивать и мутировать их, а затем отбирать лучшие для следующего поколения.



Reasoning - это способность модели выполнять многшаговые рассуждения, разложение задачи и выдачу промежуточных выводов (rationales, [chain-of-thought](https://arxiv.org/abs/2201.11903)).

Мы можем вызвать такое рассуждающее поведение у модели используя промпт ([CoT prompting](https://arxiv.org/abs/2005.14165) включающий в себя few-shot, Zero-shot CoT, [ReAct](https://arxiv.org/abs/2210.03629) (думай - делай))

Можно обучить модель делать цепочки рассуждения. 
используя на SFT этапе датасет с цепочками рассуждений. В простейшем случае это отличается от стандартного SFT этапа наличием дополнительного токена `think` или полем `rationale` в шаблоне ответа.

Затем проводится этап Instruction tuning, как некое дополнение к RLHF, на парах "инструкция -> правильное поведение".




В [DeepSeek-V3](https://arxiv.org/abs/2412.19437) MTP (multi token prediction) Вместо классической next-token prediction (предсказать только $ x_{t + 1} $ при позиции $ t $), MTP требует у модели предсказать сразу несколько следующих токенов $ x_{t+1} ​ ,..., x_{t+M} $​ (или их распределения) на каждом шаге.

Благодаря этому:
- больше градиентных сигналов на те же параметры
- модель вынуждена формировать представления, которые пригодны сразу для предсказания нескольких шагов
- потенциальное ускорение генерации и уменьшение задержки





[TransMLA](http://arxiv.org/abs/2502.07864) - Multi head latent attention 

Отлично — собрал понятный и практичный обзор метрик, которыми обычно оценивают LLM: что они измеряют, как считаются, сильные и слабые стороны, типичные применения и практические советы. На русском, коротко по делу.

# Общее замечание перед списком

Автоматические метрики редко дают «полную картину». Они полезны для быстрых A/B-сравнений и регресс-тестов, но почти всегда нужно несколько метрик + человеческая оценка для финального решения (особенно для open-ended generation, диалогов, fact-checking и т.п.).

---

# 1. Perplexity (PPL)

**Что меряет:** насколько хорошо модель предсказывает текст в среднем (обратная экспонента к пер-токеновому кросс-энтропийному loss).
**Формула (интуитивно):** $ \text{PPL} = \exp\big(\frac{1}{N}\sum_{t}-\log p(x_t\mid x_{<t})\big)$.
**Интерпретация:** меньше — лучше (модель «менее удивлена» данными).
**Плюсы:** простая, пригодна для оценки языковой согласованности и предобучения, хорошо сравнивать на одном и том же датасете/токенизации.
**Минусы:** чувствительна к токенизации (разные токенизаторы дают разные PPL), не отражает качество генерации по задачам (правильность ответа, фактичность) и плохо коррелирует с human-judgement для open-ended задач.
**Когда использовать:** language modeling, pretraining, мониторинг overfitting.

---

# 2. BLEU / SacreBLEU

**Что меряет:** n-gram совпадения между гипотезой и одним/несколькими референсами (обычно до 4-грамм) с модифицированным precision и brevity penalty.
**Интерпретация:** больше — лучше. SacreBLEU — стандартизированная реализация BLEU (репродуцируемые метрики).
**Плюсы:** простота, широко используется в машинном переводе, хорошо подходит когда референс-ориентированная задача и много референсов.
**Минусы:** нечувствителен к перефразированию и синонимам, плохо коррелирует в задачах, где допустим широкий диапазон корректных ответов (summary, open Q/A).
**Когда использовать:** машинный перевод, структурированные NLG, где референсы «узкие».

---

# 3. ROUGE (ROUGE-N, ROUGE-L)

**Что меряет:** перекрытие n-gram (ROUGE-N) и длину наибольшей общей подпоследовательности (ROUGE-L) — часто для суммаризации.
**Плюсы:** стандарт для суммаризации, прост в реализации.
**Минусы:** как и BLEU, чувствителен к лексическим совпадениям; может недооценивать хорошую абстрактную суммаризацию.
**Когда использовать:** абстрактная/экстрактивная суммаризация (в сочетании с human eval).

---

# 4. METEOR / chrF

**METEOR:** учитывает синонимы, стемминг и выравнивание — иногда коррелирует лучше с human judgment чем BLEU в небольших корпусах.
**chrF:** подсчитывает F-меру для character n-grams — полезно для агглютинативных языков.
**Когда:** альтернативы BLEU/ROUGE при специфичных языках или когда нужно учитывать морфологию.

---

# 5. BERTScore

**Что меряет:** сходство токенов на уровне эмбеддингов (BERT/RoBERTa) — сопоставляет гипотезу и референс через cosine similarity эмбеддингов.
**Плюсы:** чувствителен к синонимии и перефразированию; часто лучше коррелирует с человеческой оценкой для многих NLG задач.
**Минусы:** зависит от выбранных эмбеддингов и может «прощать» фактические ошибки (т.е. похожий семантически, но неверный факт — всё ещё высокий BERTScore).
**Когда:** суммаризация, перевод, генерация текста — как дополнение к n-gram метрикам.

---

# 6. BLEURT / COMET / BLEURT-like learned metrics

**Что это:** нейросетевые метрики, дообученные на человеческих оценках (BLEURT, COMET для MT, etc.). Выдают число, которое пытается напрямую предсказать человеческую оценку качества.
**Плюсы:** гораздо лучше коррелируют с человеческими рейтингами, учитывают семантику и качество флюэнси. COMET — ориентированная на MT, BLEURT — на более общий NLG.
**Минусы:** могут переобучаться на распределении тренировочных данных, требовательны к domain-match; «черный ящик» — меньше объяснимости.
**Когда:** когда есть потребность в автоматическом приближении human judgement (A/B тесты, фильтрация кандидатов).

---

# 7. MAUVE / Distributional metrics

**MAUVE:** пытается измерить расхождение между распределиями сгенерированного текста и реальных (human) текстов; оценивает качество/разнообразие в распределенном смысле.
**Плюсы:** полезна для обнаружения mode collapse или чрезмерной «маловероятной» генерации.
**Минусы:** интерпретация неочевидна, чувствительна к размеру и домену.
**Когда:** оценка качества генеративной модели в целом (style, diversity), особенно для open-ended generation.

---

# 8. Diversity & Degeneration: distinct-n, Self-BLEU

**distinct-n:** доля уникальных n-gram среди всех сгенерированных — измеряет разнообразие.
**Self-BLEU:** обратный показатель (насколько поколение повторяет себя) — низкая self-BLEU → высокое разнообразие.
**Когда:** полезны для диалоговых моделей и творческой генерации, чтобы измерить разнообразие и избежать повторов.

---

# 9. Task-specific metrics

* **Exact Match (EM)** и **F1** — для QA (span extraction): EM — процент полностью совпавших ответов, F1 — токен-уровневое согласие.
* **Pass\@k / pass\@1** — для code generation: вероятность, что среди k сгенерированных программ есть корректная.
* **Accuracy** — для classification / multiple-choice tasks.
* **ROUGE/BLEU/BLEURT + QA metrics** — для QG/QA pipelines комбинируют.

---

# 10. Factuality / Hallucination метрики

* **QAGS / QuestEval / QuestEval-like** — задают вопросы по.reference/hypothesis и сравнивают ответы, либо используют QA-модели, чтобы оценить соответствие.
* **FactCC / FEVER-style** — маркируют утверждения на согласованность с контекстом.
* **Model-based verifiers**: запуск fact-checker (retrieval+verification) для каждого утверждения.
  **Важно:** автоматические factuality-метрики часто дают много ложных срабатываний и требуют ручной валидации.

---

# 11. Human evaluation (неавтоматическое, но критичное)

Стандартные схемы:

* **Pairwise preference** (A vs B) — чаще всего надёжнее и проще для аннотаторов.
* **Likert шкала** (1–5) по критериям: fluency, relevance, factuality, helpfulness.
* **Best-worst scaling** — полезно для ранжирования множества систем.
  **Совет:** проводить blind A/B, пояснять инструкции аннотаторам, репликивать на разных аннотаторах и вычислять меж-ранговую согласованность.

---

# 12. Практические рекомендации (чеклист)

1. **Комбинируйте метрики.** Для summarization возьмите ROUGE + BERTScore + BLEURT/COMET + ручная проверка фактов.
2. **Сравнивайте в рамках одного токенизатора/референса** (особенно для PPL).
3. **Старайтесь измерять статистическую значимость** (bootstrap resampling, approximate randomization) при A/B.
4. **Калибруйте learned-metrics** (BLEURT/COMET) под ваш домен, если возможно.
5. **Не доверяйте одному числу**: проверьте качественные разборы ошибок (error analysis).
6. **Для open-ended**: больше веса human evaluation + distributional metrics (MAUVE) + factuality checks.
7. **Для code**: используйте pass\@k + unit tests (functional).
8. **Логи и мониторинг в продакшн:** track perplexity, toxicity scores, hallucination flags, average length, latency.

---

# 13. Как интерпретировать числа (коротко)

* **BLEU/ROUGE:** абсолютное значение мало говорит; сравнивайте системы на одном датасете.
* **BERTScore / BLEURT / COMET:** более интерпретируемы в относительном сравнении; положительная разница часто значима.
* **Perplexity:** относительная величина по датасету; снижение PPL на 10% — обычно заметно, но не гарантирует улучшение downstream.
* **Pass\@k (code):** специалисты часто смотрят pass\@1 и pass\@10, потому что генерация нескольких кандидатов — обычная практика.

---

Если нужно, могу:

* предложить **комбинацию метрик** для вашей конкретной задачи (скажите: перевод / суммаризация / диалог / код / QA),
* подготовить **скрипт на Python** (HF + sacrebleu + bert\_score + bleurt + mauve) для оценки ваших выводов,
* или составить план **human evaluation** (инструкции для аннотаторов, шаблон анкеты, как анализировать результаты).

Что из этого сделать дальше?
